In [140]:
#from google.colab import  drive
#drive.mount('/drive')

In [141]:
import pandas as pd
import re
from time import sleep
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
import gc

In [142]:
# Interprération d'une date en format "YYYYMMDD"
def parse_date(_date):
    annee = _date[0:4]
    mois = _date[4:6]
    jour = _date[6:8]
    if mois == '00':
        mois = '07'
    if jour == '00':
        jour = '01'
    if annee == '0000':
        return None
    try:
      return datetime(int(annee), int(mois), int(jour))
    except ValueError as e:
      if e.args[0] == "day is out of range for month":
        return datetime(int(annee), int(mois), int(jour)-3)
      return None

In [143]:
# Calcul de l'age révolu à la date du décès
def age_revolu(date_naissance, date_deces):
    _date_naissance = parse_date(date_naissance)
    _date_deces = parse_date(date_deces)
    if _date_naissance and _date_deces:
        _age_revolu = int((_date_deces - _date_naissance).days/365)
        if _age_revolu >= 100:
          _age_revolu = 100
        return _age_revolu, _date_naissance, _date_deces
    else:
        return None, None, None

In [144]:
# Expression régulière pour interpréter une ligne du fichier de décès INSEE
_regexp = r'([A-Z\s\-\'\*]+)\/\s*([12])([0-9]{8})([0-9AB]{5})*(.+[^0-9])([0-9]{8})([0-9AB]{5})*'
_pattern = re.compile(_regexp)

In [145]:
def parse(_string, annee_fichier):
    _match = _pattern.match(_string[0:167])
    if _match:
        _age_revolu, _date_naissance, _date_deces = age_revolu(_match[3], _match[6])
        return {'NOM_PRENOMS': _match[1],
                'SEXE': "H" if _match[2] == "1" else "F",
                'DATE_NAISSANCE': _date_naissance,
                'COMM_NAISSANCE': _match[4],
                'LIEU_NAISSANCE': _match[5],
                'DATE_DECES': _date_deces,
                'COMM_DECES': _match[7],
                'AGE_REVOLU': _age_revolu,
                'ANNEE': int(_match[6][0:4]),
                'ORIG': None,
                'ANNEE_FICHIER': int(annee_fichier)
               }
    return {'ORIG': _string, 'ANNEE_FICHIER': annee_fichier}

In [146]:
# Exemple
parse('OSWALD*ANNA/                                                                    21928030599109LORRACH                       ALLEMAGNE                     201912071410020                ', 2020)

{'AGE_REVOLU': 91,
 'ANNEE': 2019,
 'ANNEE_FICHIER': 2020,
 'COMM_DECES': '14100',
 'COMM_NAISSANCE': '99109',
 'DATE_DECES': datetime.datetime(2019, 12, 7, 0, 0),
 'DATE_NAISSANCE': datetime.datetime(1928, 3, 5, 0, 0),
 'LIEU_NAISSANCE': 'LORRACH                       ALLEMAGNE                     ',
 'NOM_PRENOMS': 'OSWALD*ANNA',
 'ORIG': None,
 'SEXE': 'F'}

In [147]:
def k_format(_float):
  return f'{int(_float/100)/10}'

In [148]:
def free(_object):
  del _object
  gc.collect()

In [149]:
# https://www.data.gouv.fr/fr/datasets/fichier-des-personnes-decedees/
fichiers_insee_deces = {
    2021.02: 'https://www.data.gouv.fr/fr/datasets/r/4859e2c2-8b7b-43fc-9fc0-cc87bb3efa04',
    2021.01: 'https://www.data.gouv.fr/fr/datasets/r/9bc3b4b0-faf1-49cd-bd1f-feb5bec303bd',
    2020: 'https://www.data.gouv.fr/fr/datasets/r/a1f09595-0e79-4300-be1a-c97964e55f05',
    2019: 'https://www.data.gouv.fr/fr/datasets/r/02acf8f5-9190-4f8e-a37c-3b34eccac833',
    2018: 'https://www.data.gouv.fr/fr/datasets/r/c2a97b38-5c0d-4f21-910f-1cea164c2c89',
    2017: 'https://www.data.gouv.fr/fr/datasets/r/fd61ff96-1e4e-450f-8648-3e3016edbe34',
    2016: 'https://www.data.gouv.fr/fr/datasets/r/8fb032c1-b81e-46c4-a48a-15380ce41e40',
    2015: 'https://www.data.gouv.fr/fr/datasets/r/22274343-816c-4220-8ca9-05ac0ff9b6f8',
    2014: 'https://www.data.gouv.fr/fr/datasets/r/9409dd78-1c54-47a4-850c-d903bb274a32',
    2013: 'https://www.data.gouv.fr/fr/datasets/r/33769f81-6507-4b0e-8f7a-f078a2c47084',
    2012: 'https://www.data.gouv.fr/fr/datasets/r/85a225fc-f0ab-4462-b8be-03981332bb98',
    2011: 'https://www.data.gouv.fr/fr/datasets/r/2581b087-003e-4b21-8175-7e7eef38c9cb',
#    2010: 'https://www.data.gouv.fr/fr/datasets/r/2a7d69bd-6edf-4181-8b96-3ac4f0552ba6',
#    2009: 'https://www.data.gouv.fr/fr/datasets/r/240f89af-1b80-4c48-b896-d4f4ae943d4a',
#    2008: 'https://www.data.gouv.fr/fr/datasets/r/4d887ca2-af0e-4407-aa19-b7ab12077223',
#    2007: 'https://www.data.gouv.fr/fr/datasets/r/7709f33d-4624-441a-bc04-bd72a28e4d08',
#    2006: 'https://www.data.gouv.fr/fr/datasets/r/dd9ca2d6-21bb-40f1-a26c-9c50a3aceeef',
#    2005: 'https://www.data.gouv.fr/fr/datasets/r/045bcee9-bb3c-4410-a013-f2a4183473fc',
#    2004: 'https://www.data.gouv.fr/fr/datasets/r/36653f18-ae52-40c8-9c5f-6de43eeddac6'
}
annee_min = min(fichiers_insee_deces.keys()) + 1

In [150]:
# Récupration et lecture des fichiers de décès INSEE
_dataframes = {annee_fichier: pd.read_csv(_fichier, sep='@@@', engine='python', header=None, encoding='latin1')
               for annee_fichier, _fichier in fichiers_insee_deces.items()}

In [151]:
# Concatenation et elimination des quelques lignes dont les informations sont inexploitables
deces = pd.concat([pd.DataFrame([y for y in df[0].map(lambda x: parse(x, annee_fichier))])
                   for annee_fichier, df in _dataframes.items()])
free(_dataframes)

In [152]:
#deces.to_parquet('/drive/My Drive/Colab Notebooks/deces.parquet')

In [153]:
#deces = pd.read_parquet('/drive/My Drive/Colab Notebooks/deces.parquet')

In [154]:
['Fichier {} : {} décès dont {}% en France métropolitaine et {} inexploitables'.format
 (annee_fichier,
  k_format(deces.query(f'ANNEE_FICHIER=={int(annee_fichier)}').shape[0]),
  int(10000*deces[~deces['COMM_DECES'].fillna('').str.startswith('99')].query(f'ANNEE_FICHIER=={int(annee_fichier)}').shape[0]/deces.query(f'ANNEE_FICHIER=={int(annee_fichier)}').shape[0])/100,
  deces.query(f'ANNEE_FICHIER=={int(annee_fichier)} and not (AGE_REVOLU==AGE_REVOLU and SEXE==SEXE and ANNEE==ANNEE)').shape[0],
)
for annee_fichier in fichiers_insee_deces.keys()]

['Fichier 2021.02 : 125.1 décès dont 99.09% en France métropolitaine et 0 inexploitables',
 'Fichier 2021.01 : 125.1 décès dont 99.09% en France métropolitaine et 0 inexploitables',
 'Fichier 2020 : 679.9 décès dont 99.1% en France métropolitaine et 3 inexploitables',
 'Fichier 2019 : 625.3 décès dont 98.88% en France métropolitaine et 1 inexploitables',
 'Fichier 2018 : 620.1 décès dont 98.81% en France métropolitaine et 0 inexploitables',
 'Fichier 2017 : 612.9 décès dont 98.74% en France métropolitaine et 1 inexploitables',
 'Fichier 2016 : 603.3 décès dont 98.67% en France métropolitaine et 1 inexploitables',
 'Fichier 2015 : 609.6 décès dont 98.0% en France métropolitaine et 1 inexploitables',
 'Fichier 2014 : 569.4 décès dont 99.01% en France métropolitaine et 1 inexploitables',
 'Fichier 2013 : 582.6 décès dont 98.94% en France métropolitaine et 1 inexploitables',
 'Fichier 2012 : 579.9 décès dont 98.85% en France métropolitaine et 0 inexploitables',
 'Fichier 2011 : 549.1 décès

In [155]:
# Limitation au décès survenus en France metropolitaine
_cols = ['SEXE', 'DATE_NAISSANCE', 'COMM_NAISSANCE', 'DATE_DECES',	'COMM_DECES',	'AGE_REVOLU',	'ANNEE', 'ANNEE_FICHIER']
deces_metro = deces[~deces['COMM_DECES'].fillna('').str.startswith('99')].query('AGE_REVOLU==AGE_REVOLU and SEXE==SEXE and ANNEE==ANNEE')[_cols]

In [156]:
free(deces)

In [157]:
# Jusqu'à 4% des décès metropolitains d'une année sont remontés dans les fichiers des années suivantes
for annee in range(annee_min-1, 2021):
    deces_annee_n = deces_metro.query(f'DATE_DECES>="{annee}-01-01" and DATE_DECES<="{annee}-12-31"')
    fichier_n_plus = deces_annee_n.query(f'ANNEE_FICHIER>={annee+1}')['DATE_DECES'].count()
    print(annee, int(10000*(fichier_n_plus/deces_annee_n.shape[0]))/100)

2011 3.48
2012 3.28
2013 2.87
2014 2.54
2015 2.04
2016 2.22
2017 2.67
2018 2.55
2019 2.2
2020 1.64


In [158]:
# Au moins 99.6% des décès metropolitains sont remontés dans les deux mois suivant la date du décès
for annee in range(annee_min, 2020):
    deces_annee_n_hors_decembre = deces_metro.query(f'DATE_DECES>="{annee-1}-11-01" and DATE_DECES<"{annee}-11-01"')
    fichier_n_plus = deces_annee_n_hors_decembre.query(f'ANNEE_FICHIER>={annee+1}')['DATE_DECES'].count()
    print(annee, int(10000*(1 - fichier_n_plus/deces_annee_n_hors_decembre.shape[0]))/100)

2012 99.44
2013 99.64
2014 99.71
2015 99.87
2016 99.84
2017 99.74
2018 99.85
2019 99.91


In [159]:
_dict = pd.Series({annee: datetime(2020, 12, 31) + timedelta(days=-n*365) for n, annee in enumerate(range(2020, annee_min-2, -1))}).sort_index()
def annee_glissante(_date):
  for annee, date_limite in _dict.items():
    if _date <= date_limite:
      return annee

In [160]:
# Re-definition de l'année de décès pour avoir des années glissantes de 365 jours (corrige le biais des anées bissextiles)
deces_metro['ANNEE'] = deces_metro['DATE_DECES'].map(annee_glissante)

In [191]:
f = deces_metro.query('SEXE=="F"').groupby('ANNEE').ANNEE_FICHIER.count().map(k_format).reset_index().rename(columns={'ANNEE_FICHIER': 'F'})
h = deces_metro.query('SEXE=="H"').groupby('ANNEE').ANNEE_FICHIER.count().map(k_format).reset_index().rename(columns={'ANNEE_FICHIER': 'H'})
f.merge(h, on="ANNEE").sort_values('ANNEE', ascending=False)

,ANNEE,F,H
9,2020.0,333.6,335.2
8,2019.0,309.4,306.1
7,2018.0,306.7,305.0
6,2017.0,306.0,301.9
5,2016.0,296.6,296.8
4,2015.0,296.9,297.2
3,2014.0,277.8,283.7
2,2013.0,282.6,288.7
1,2012.0,282.9,287.9
0,2011.0,283.5,297.1


In [162]:
source_insee = 'https://www.insee.fr/fr/statistiques/pyramide/3312958/xls/pyramides-des-ages_bilan-demo_2019.xls'
annees = range(annee_min, 2021)
pyramide_metro_orig = {}
pyramide_metro_orig['H'] = pd.read_excel(source_insee, sheet_name='France métropolitaine', skiprows=[x for x in range(0,10) if x != 8], nrows=106)[annees].reset_index()
pyramide_metro_orig['F'] = pd.read_excel(source_insee, sheet_name='France métropolitaine', skiprows=[x for x in range(0,119) if x != 8], nrows=106)[annees].reset_index()

In [163]:
pyramide_metro = pd.concat([
                            pd.DataFrame([{'ANNEE': annee, 'SEXE': sexe, 'NB': nb, 'AGE_REVOLU': index} for (index, nb) in enumerate(pyramide_metro_orig[sexe][annee])])
                            for annee in annees for sexe in ['H', 'F']])

centenaires = pyramide_metro.query('AGE_REVOLU>=100').groupby(['ANNEE', 'SEXE'])['NB'].sum().reset_index()
centenaires['AGE_REVOLU'] = 100
pyramide_metro = pd.concat([pyramide_metro.query('AGE_REVOLU < 100'), centenaires])

In [164]:
#pyramide_metro.to_parquet('/drive/My Drive/Colab Notebooks/pyramide_metro.parquet')

In [165]:
# Aggrégation des décès par année, age révolu et sexe
deces_metro_agg = deces_metro.groupby(['AGE_REVOLU', 'ANNEE', 'SEXE'])['DATE_DECES'].count().reset_index()
deces_metro_agg.columns = ['AGE_REVOLU', 'ANNEE', 'SEXE', 'NB_DECES']

In [166]:
# Jointure entre les pyrmides d'âge et les décès par age révolu
_merge = deces_metro_agg.merge(pyramide_metro)

In [167]:
# Retourne les taux de mortalité par age révolu pour une année et un sexe donnés
def mortalite_par_age(df, annee, sexe):
    _filtre = df['ANNEE'] == annee
    if sexe != "HF":
      _filtre = _filtre & (df['SEXE']==sexe)
    _df = df[_filtre]
    _df = _df.groupby('AGE_REVOLU')
    return pd.DataFrame(_df['NB_DECES'].sum() / _df['NB'].sum(), columns=[f'{annee}_{sexe}_MORTALITE'])

In [168]:
# Calcul des series de taux de mortalité pour toutes les combinaisons d'année et sexe
taux_mortalite = pd.concat([mortalite_par_age(_merge, annee, sexe)
                            for annee in range(annee_min, 2021)
                            for sexe in ["H", "F", "HF"]], axis=1)

In [169]:
# Enrichissement avec la pyramide d'age 2020 et la mortalité max et min observée entre 2017 et 2019
for sexe in ["H", "F", "HF"]:
    taux_mortalite[f'NB_2020_{sexe}'] = _merge.query('ANNEE==2020').query('SEXE==SEXE' if sexe == "HF" else f'SEXE=="{sexe}"').groupby('AGE_REVOLU')['NB'].sum()
    taux_mortalite[f'MAX_{sexe}_MORTALITE'] =taux_mortalite[[f'{annee}_{sexe}_MORTALITE'
                                                             for annee in range(annee_min, 2020)]].max(axis=1)
    taux_mortalite[f'MIN_{sexe}_MORTALITE'] = taux_mortalite[[f'{annee}_{sexe}_MORTALITE'
                                                              for annee in range(annee_min, 2020)]].min(axis=1)
    taux_mortalite[f'CENT75_{sexe}_MORTALITE'] =taux_mortalite[[f'{annee}_{sexe}_MORTALITE'
                                                             for annee in range(annee_min, 2020)]].quantile(0.75, axis=1)
    taux_mortalite[f'CENT25_{sexe}_MORTALITE'] = taux_mortalite[[f'{annee}_{sexe}_MORTALITE'
                                                              for annee in range(annee_min, 2020)]].quantile(0.25, axis=1)
    taux_mortalite[f'MED_{sexe}_MORTALITE'] = taux_mortalite[[f'{annee}_{sexe}_MORTALITE'
                                                              for annee in range(annee_min, 2020)]].quantile(0.5, axis=1)
    taux_mortalite[f'MOY_{sexe}_MORTALITE'] = taux_mortalite[[f'{annee}_{sexe}_MORTALITE'
                                                              for annee in range(annee_min, 2020)]].mean(axis=1)

In [175]:
_list = []
for sexe in ["F", "H", "HF"]:
  for ages in [(0, 44), (45, 64), (65,74), (75, 89), (90, 100), (0, 64), (65, 100)]:
    _pop = taux_mortalite.query(f'AGE_REVOLU>={ages[0]} and AGE_REVOLU<={ages[1]}')[f'NB_2020_{sexe}']
    _deces_2020 = _pop.multiply(taux_mortalite[f'2020_{sexe}_MORTALITE']).sum()
    _enveloppe_25 = _pop.multiply(taux_mortalite[f'CENT25_{sexe}_MORTALITE']).sum()
    _enveloppe_min = _pop.multiply(taux_mortalite[f'MIN_{sexe}_MORTALITE']).sum()
    _enveloppe_moy = _pop.multiply(taux_mortalite[f'MOY_{sexe}_MORTALITE']).sum()
    _enveloppe_med = _pop.multiply(taux_mortalite[f'MED_{sexe}_MORTALITE']).sum()
    _enveloppe_75 = _pop.multiply(taux_mortalite[f'CENT75_{sexe}_MORTALITE']).sum()
    _enveloppe_max = _pop.multiply(taux_mortalite[f'MAX_{sexe}_MORTALITE']).sum()
    _depassement = round(100000*(_deces_2020 - _enveloppe_max)/_enveloppe_max,2)
    _depassement_75 = round(100000*(_deces_2020 - _enveloppe_75)/_enveloppe_75,2)
    _ecart_moy = round(100000*(_deces_2020 - _enveloppe_moy)/_enveloppe_moy,2)
    _ecart_med = round(100000*(_deces_2020 - _enveloppe_med)/_enveloppe_med,2)
    _list.append(
        {'SEXE': sexe,#'Hommes' if sexe=="H" else 'Femmes',
         'AGE RÉVOLU': f'{ages[0]} à {ages[1]}',
         'DÉCÈS 2020 (en milliers)': f'{k_format(_deces_2020)}',
         #f'ENVELOPPE MIN-MAX DEPUIS {annee_min} (en pourcent)': f'{k_format(_enveloppe_min)} à {k_format(_enveloppe_max)}',
         f'ENVELOPPE CENTILE 25 à 75 DEPUIS {annee_min} (en milliers)': f'{k_format(_enveloppe_25)} à {k_format(_enveloppe_75)}',
         f'ECART MÉDIANE DEPUIS {annee_min} (en pourcent)':  f'{k_format(_ecart_med)}',
         f'ECART MOYENNE DEPUIS {annee_min} (en pourcent)':  f'{k_format(_ecart_moy)}',
         #f'ECART MAX DEPUIS {annee_min} (en pourcent)': f'{k_format(_depassement)}',
         f'ECART CENTILE 75 DEPUIS {annee_min} (en pourcent)': f'{k_format(_depassement_75)}'
         })

In [176]:
pd.DataFrame(_list)

,SEXE,AGE RÉVOLU,DÉCÈS 2020 (en milliers),ENVELOPPE CENTILE 25 à 75 DEPUIS 2012 (en milliers),ECART MÉDIANE DEPUIS 2012 (en pourcent),ECART MOYENNE DEPUIS 2012 (en pourcent),ECART CENTILE 75 DEPUIS 2012 (en pourcent)
0,F,0 à 44,6.5,6.5 à 7.2,-4.9,-4.9,-9.3
1,F,45 à 64,27.8,27.3 à 28.5,-0.4,-0.2,-2.2
2,F,65 à 74,36.9,34.4 à 35.9,5.4,4.4,2.8
3,F,75 à 89,132.4,127.5 à 134.4,1.4,1.0,-1.4
4,F,90 à 100,129.8,125.0 à 135.0,-0.2,-3.1,-3.8
5,F,0 à 64,34.4,33.8 à 35.7,-1.3,-1.2,-3.6
6,F,65 à 100,299.2,287.0 à 305.4,1.2,-0.4,-2.0
7,H,0 à 44,12.8,12.8 à 13.9,-3.9,-4.5,-8.3
8,H,45 à 64,53.0,53.5 à 57.0,-4.2,-4.2,-6.9
9,H,65 à 74,67.6,62.9 à 66.4,5.0,3.7,1.8


In [174]:
import plotly.graph_objects as go
fig = go.Figure()
fig.update_layout(
    title='Taux de mortalité par âge révolu',
    font=dict(
        family="Century Gothic",
        size=18,
    ),
    legend=dict(
      yanchor="top", y=0.99, xanchor="left", x=0.01
    )
)
sexe = 'H'
col = f'CENT25_{sexe}_MORTALITE'
fig.add_trace(go.Scatter(x=taux_mortalite[col].index, y=taux_mortalite[col], name='2012_2019_'+col, line_color='grey'))
col = f'CENT75_{sexe}_MORTALITE'
fig.add_trace(go.Scatter(x=taux_mortalite[col].index, y=taux_mortalite[col], name='2012_2019_'+col, line_color='grey', fill='tonexty'))
col = f'2020_{sexe}_MORTALITE'
fig.add_trace(go.Scatter(x=taux_mortalite[col].index, y=taux_mortalite[col], name=col, line_color='red'))

fig.show()

In [173]:
import plotly.graph_objects as go
fig = go.Figure()
fig.update_layout(
    title='Taux de mortalité par âge révolu',
    font=dict(
        family="Century Gothic",
        size=18,
    ),
    legend=dict(
      yanchor="top", y=0.99, xanchor="left", x=0.01
    )
)
sexe = 'H'
col = f'MIN_{sexe}_MORTALITE'
fig.add_trace(go.Scatter(x=taux_mortalite[col].index, y=taux_mortalite[col], name='2012_2019_'+col, line_color='grey'))
col = f'MAX_{sexe}_MORTALITE'
fig.add_trace(go.Scatter(x=taux_mortalite[col].index, y=taux_mortalite[col], name='2012_2019_'+col, line_color='grey', fill='tonexty'))
col = f'2020_{sexe}_MORTALITE'
fig.add_trace(go.Scatter(x=taux_mortalite[col].index, y=taux_mortalite[col], name=col, line_color='red'))
fig.show()